# Exploring the Galaxy Toys Example

We are now going to explore the Galaxy Toys example a bit more. Let's first see how to create and solve the model using Gurobi. Then, how can we get the sensitivity analysis information out?

## Formulation 

Recall that our formulation looked like this:

| | | |
| --- | --- | --- |
| Let | | |
| $x_{s}$ | = | number of lots (dozens) of Space Rays to produce next week |
| $x_{p}$ | = | number of lots (dozens) of Phasers to produce next week |

| | | | | | | |
| --- | --- | --- | --- | --- | --- | --- |
| $\max$ | $8x_{s}$ | $+$ | $5x_{p}$ | | | |
| s.t. | $2x_{s}$ | $+$ | $1x_{p}$ | $\le$ | $1200$ | {plastic pounds} |
| | $3x_{s}$ | $+$ | $4x_{p}$ | $\le$ | $2400$ | {minutes of production} |
| | $1x_{s}$ | $+$ | $1x_{p}$ | $\le$ | $800$ | {overall production limit} |
| | $1x_{s}$ | $-$ | $1x_{p}$ | $\le$ | $450$ | {mix of products produced} |
| | $x_{s}$ | | | $\ge$ | $0$ | {non-negativity} |
| | | | $x_{p}$ | $\ge$ | $0$ | {non-negativity} |

In [31]:
# import necessary modules/packages
import gurobipy as gp
from gurobipy import GRB

import pandas as pd

In [32]:
####                ####
#    ORIGINAL MODEL    #
####                ####
# Create the model object
m = gp.Model('galaxy_toys')

# Specify how to optimize and time limit (seconds)
m.ModelSense = GRB.MAXIMIZE

# You can set the time limit for the solving
# unnecessary for this small problem
#m.setParam('TimeLimit', 600)

# Create decision variables
# We tell the solver that the variables are continuous,
#   their names, and their lower bounds
x_s = m.addVar(vtype=GRB.CONTINUOUS, name='space_rays', lb=0.0)
x_p = m.addVar(vtype=GRB.CONTINUOUS, name='phasers', lb=0.0)

# Add the objective function
m.setObjective(8 * x_s + 5 * x_p)

# Add the constraints
# We can simply write out the constraints for the first parameter
# The second parameter names the constraint
m.addConstr(2*x_s + x_p <= 1200, name='plastic')
m.addConstr(3*x_s + 4*x_p <= 2400, name='labor')
m.addConstr(x_s + x_p <= 800, name='total_production')
m.addConstr(x_s - x_p <= 450, name='product_mix')

# update the model
m.update()

# solve
m.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 24.04.2 LTS")

CPU model: Intel(R) Core(TM) Ultra 7 155H, instruction set [SSE2|AVX|AVX2]
Thread count: 11 physical cores, 22 logical processors, using up to 22 threads

Optimize a model with 4 rows, 2 columns and 8 nonzeros
Model fingerprint: 0xbb372ecc
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [5e+00, 8e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+02, 2e+03]
Presolve time: 0.01s
Presolved: 4 rows, 2 columns, 8 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.3000000e+31   5.250000e+30   1.300000e+01      0s
       4    5.0400000e+03   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.01 seconds (0.00 work units)
Optimal objective  5.040000000e+03


## Getting the Results

We now want to get the results and print them in a little nicer format. 

In [33]:
# Get the results out
print(f'To generate the optimal profit of ${m.ObjVal:0.2f}, you should produce the following amounts:')
for v in m.getVars():
    print(f'   {v.VarName} = {v.X}')

To generate the optimal profit of $5040.00, you should produce the following amounts:
   space_rays = 480.0
   phasers = 240.0


## Senstivity Analysis - Variables

We are interested in the finding the reduced cost and range of optimality for each variable. To help us with this, I have created a function that will take in the model object and return a `pandas` `DataFrame` for easier readability.

In [34]:
# define a function to return the sensitivity analysis for the variables
def get_SA_vars(the_model):
    ''' 
    This is a helper function that collects all the sensitivity analysis 
    for the "variable section" that you would see in the sensitivity
    report from Excel and returns it a as pandas DataFrame.

    the_model : an instance of gp.Model 

    returns a pandas DataFrame
    '''
    var_sensitivity =[]
    for v in the_model.getVars():
        var_sensitivity.append([v.VarName, v.X, v.RC, v.Obj, v.SAObjLow, v.SAObjUp])

    retValue = pd.DataFrame(var_sensitivity)
    retValue.columns = ['variable', 'final_value', 'reduced_cost', 'obj_fn_coeff', 'range_opt_low', 'range_opt_up']

    return retValue

In [35]:
# see if function works
get_SA_vars(m)

,variable,final_value,reduced_cost,obj_fn_coeff,range_opt_low,range_opt_up
0,space_rays,480.0,0.0,8.0,3.75,10.000000
1,phasers,240.0,0.0,5.0,4.00,10.666667


## Changing the Objective Function Coefficient for Space Rays

Let's go ahead and change the objective function coefficient from the original \\$8 to \\$7 and re-run the solver to see what we get. If we understand what the range of optimality means, then you should be able to tell exactly what the result will be without having to solve the model. What is it?

In [36]:
# Change the objective function
m.setObjective(7 * x_s + 5 * x_p)
m.update()
m.optimize()

# get the results
print(f'To generate the optimal profit of ${m.ObjVal:0.2f}, you should produce the following amounts:')
for v in m.getVars():
    print(f'   {v.VarName} = {v.X}')

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 24.04.2 LTS")

CPU model: Intel(R) Core(TM) Ultra 7 155H, instruction set [SSE2|AVX|AVX2]
Thread count: 11 physical cores, 22 logical processors, using up to 22 threads



Optimize a model with 4 rows, 2 columns and 8 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [5e+00, 7e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+02, 2e+03]
LP warm-start: use basis

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.5600000e+03   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  4.560000000e+03
To generate the optimal profit of $4560.00, you should produce the following amounts:
   space_rays = 480.0
   phasers = 240.0


In [37]:
# Look at sensitivity analysis for variables again
get_SA_vars(m)

,variable,final_value,reduced_cost,obj_fn_coeff,range_opt_low,range_opt_up
0,space_rays,480.0,0.0,7.0,3.75,10.000000
1,phasers,240.0,0.0,5.0,3.50,9.333333


## Understanding Reduced Costs

Both reduced costs were \\$0 because both variables were greater than zero. If we change the objective function coefficient for space rays to \\$2 and re-solve, we should see a reduced cost other than \\$0 show up. Let's try it.

In [38]:
# Change the objective function
m.setObjective(2 * x_s + 5 * x_p)
m.update()
m.optimize()

# get the results
print(f'To generate the optimal profit of ${m.ObjVal:0.2f}, you should produce the following amounts:')
for v in m.getVars():
    print(f'   {v.VarName} = {v.X}')

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 24.04.2 LTS")

CPU model: Intel(R) Core(TM) Ultra 7 155H, instruction set [SSE2|AVX|AVX2]
Thread count: 11 physical cores, 22 logical processors, using up to 22 threads

Optimize a model with 4 rows, 2 columns and 8 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [2e+00, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+02, 2e+03]
LP warm-start: use basis

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.8000000e+30   1.600000e+30   2.800000e+00      0s
       1    3.0000000e+03   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds (0.00 work units)
Optimal objective  3.000000000e+03
To generate the optimal profit of $3000.00, you should produce the following amounts:
   space_rays = 0.0
   phasers = 600.0


In [39]:
# Look at sensitivity analysis for variables again
get_SA_vars(m)

,variable,final_value,reduced_cost,obj_fn_coeff,range_opt_low,range_opt_up
0,space_rays,0.0,-1.75,2.0,-inf,3.75
1,phasers,600.0,0.00,5.0,2.666667,inf


This means we would need to make the profit for space rays at least \\$2 + \\$1.75 = \\$3.75 before it becomes economically feaisble to produce space rays. Another way to think of the reduced cost is as follows. If you leave the profit for space rays at \\$2 and were forced to produce one lot/unit/dozen, what effect would that have on the profit? You can add an additional constraint $s \geq 1$ and re-solve.

In [40]:
# look at the current model
m.display()

Maximize
  2.0 space_rays + 5.0 phasers
Subject To
  plastic: 2.0 space_rays + phasers <= 1200
  labor: 3.0 space_rays + 4.0 phasers <= 2400
  total_production: space_rays + phasers <= 800
  product_mix: space_rays + -1.0 phasers <= 450


/tmp/ipykernel_5706/70023011.py:2: DeprecationWarning: Model.display() is deprecated
  m.display()


In [41]:
# Add forced production of 1 unit of space rays
m.addConstr(x_s >= 1, name='force_space_rays')
m.update()

In [42]:
m.display()

Maximize
  2.0 space_rays + 5.0 phasers
Subject To
  plastic: 2.0 space_rays + phasers <= 1200
  labor: 3.0 space_rays + 4.0 phasers <= 2400
  total_production: space_rays + phasers <= 800
  product_mix: space_rays + -1.0 phasers <= 450
  force_space_rays: space_rays >= 1


/tmp/ipykernel_5706/137993151.py:1: DeprecationWarning: Model.display() is deprecated
  m.display()


In [43]:
m.optimize()

# get the results
print(f'To generate the optimal profit of ${m.ObjVal:0.2f}, you should produce the following amounts:')
for v in m.getVars():
    print(f'   {v.VarName} = {v.X}')

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 24.04.2 LTS")

CPU model: Intel(R) Core(TM) Ultra 7 155H, instruction set [SSE2|AVX|AVX2]
Thread count: 11 physical cores, 22 logical processors, using up to 22 threads

Optimize a model with 5 rows, 2 columns and 9 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [2e+00, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+03]
LP warm-start: use basis

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.0000000e+03   1.000000e+00   0.000000e+00      0s
       1    2.9982500e+03   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.998250000e+03
To generate the optimal profit of $2998.25, you should produce the following amounts:
   space_rays = 1.0
   phasers = 599.25


## Sensitivity Analysis for RHS of Constraints

**Any** change in the right-hand-side value of a binding constraint will change the optimal solution. The optimal solution is not affected by changes in the RHS value of a non-binding constraint as long as the change is less than the slack or surplus and it is the only change made.

We often want to answer the following questions:

1. All else equal, how much would the optimal profit change if the RHS of a constraint is changed by a single unit?
2. For how many additional or fewer units will this per unit change be valid?

We can answer those questions with sensitivity analysis for the constraints. In particular, we are interested in the shadow price and the range of feasibility. The shadow price answers the first question above. The range of feasibility answers the second question. Within the range of feasibility, the same binding constraints that determine the optimal solution will remain the same. Recall, however, that the coordinates of the optimal solution (point) will change and the value of the objective function will also change.

The code cell below defines a function to get the sensitivity analysis for the constraints into a easier to read format.

In [44]:
# define a function to return the sensitivity analysis for the variables
def get_SA_constraints(the_model):
    constr_sensitivity = []
    for c in the_model.getConstrs():
        constr_sensitivity.append([c.constrName, c.RHS, the_model.getRow(c).getValue(), c.Slack, c.pi, c.SARHSLow, c.SARHSUp])

    retValue = pd.DataFrame(constr_sensitivity)
    retValue.columns = ['constraint', 'RHS', 'final_value', 'slack', 'shadow_price', 'range_feasibility_low', 'range_feasibility_up']

    return retValue

In [45]:
# Now go back to the original model
####                ####
#    ORIGINAL MODEL    #
####                ####
# Create the model object
m = gp.Model('galaxy_toys')

# Specify how to optimize and time limit (seconds)
m.ModelSense = GRB.MAXIMIZE

# You can set the time limit for the solving
# unnecessary for this small problem
#m.setParam('TimeLimit', 600)

# Create decision variables
# We tell the solver that the variables are continuous,
#   their names, and their lower bounds
x_s = m.addVar(vtype=GRB.CONTINUOUS, name='space_rays', lb=0.0)
x_p = m.addVar(vtype=GRB.CONTINUOUS, name='phasers', lb=0.0)

# Add the objective function
m.setObjective(8 * x_s + 5 * x_p)

# Add the constraints
# We can simply write out the constraints for the first parameter
# The second parameter names the constraint
m.addConstr(2*x_s + x_p <= 1200, name='plastic')
m.addConstr(3*x_s + 4*x_p <= 2400, name='labor')
m.addConstr(x_s + x_p <= 800, name='total_production')
m.addConstr(x_s - x_p <= 450, name='product_mix')

# update the model
m.update()

# solve
m.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 24.04.2 LTS")

CPU model: Intel(R) Core(TM) Ultra 7 155H, instruction set [SSE2|AVX|AVX2]
Thread count: 11 physical cores, 22 logical processors, using up to 22 threads

Optimize a model with 4 rows, 2 columns and 8 nonzeros
Model fingerprint: 0xbb372ecc
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [5e+00, 8e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+02, 2e+03]
Presolve time: 0.00s
Presolved: 4 rows, 2 columns, 8 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.3000000e+31   5.250000e+30   1.300000e+01      0s
       4    5.0400000e+03   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.01 seconds (0.00 work units)
Optimal objective  5.040000000e+03


In [46]:
m.display()

Maximize
  8.0 space_rays + 5.0 phasers
Subject To
  plastic: 2.0 space_rays + phasers <= 1200
  labor: 3.0 space_rays + 4.0 phasers <= 2400
  total_production: space_rays + phasers <= 800
  product_mix: space_rays + -1.0 phasers <= 450


/tmp/ipykernel_5706/137993151.py:1: DeprecationWarning: Model.display() is deprecated
  m.display()


In [47]:
# get the results
print(f'To generate the optimal profit of ${m.ObjVal:0.2f}, you should produce the following amounts:')
for v in m.getVars():
    print(f'   {v.VarName} = {v.X}')

To generate the optimal profit of $5040.00, you should produce the following amounts:
   space_rays = 480.0
   phasers = 240.0


In [48]:
# Get sensistivity for constraints
get_SA_constraints(m)

,constraint,RHS,final_value,slack,shadow_price,range_feasibility_low,range_feasibility_up
0,plastic,1200.0,1200.0,0.0,3.4,600.0,1350.0
1,labor,2400.0,2400.0,0.0,0.4,2050.0,2800.0
2,total_production,800.0,720.0,80.0,0.0,720.0,inf
3,product_mix,450.0,240.0,210.0,0.0,240.0,inf


### Changing RHS of Constraint

Let's change the RHS of the plastic constraint by adding one additional pound. What will the resulting objective function value?

The easiest way to do this is get the constraint by name and then change its right-hand-side value.

In [49]:
# We can pull out a specific constraint by its name
# Let's get plastic and store it a Python variable named plastic
plastic = m.getConstrByName('plastic')
print(plastic.rhs)

1200.0


In [50]:
# Now update the RHS with one additional pound of plastic
plastic.rhs = 1201

# Update the model and solve it
m.update()
m.optimize()

# get the results
print(f'To generate the optimal profit of ${m.ObjVal:0.2f}, you should produce the following amounts:')
for v in m.getVars():
    print(f'   {v.VarName} = {v.X}')

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 24.04.2 LTS")

CPU model: Intel(R) Core(TM) Ultra 7 155H, instruction set [SSE2|AVX|AVX2]
Thread count: 11 physical cores, 22 logical processors, using up to 22 threads

Optimize a model with 4 rows, 2 columns and 8 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [5e+00, 8e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+02, 2e+03]
LP warm-start: use basis

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.0434000e+03   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  5.043400000e+03
To generate the optimal profit of $5043.40, you should produce the following amounts:
   space_rays = 480.8
   phasers = 239.39999999999998


## Multiple Optimal Solutions

Can we find multiple optimal solutions? We saw one case where both the reduced cost and the value of the decision variable was 0 indicating that perhaps multiple optimal solutions existed. The second case was not like that. Let's examine the second case because it seems more interesting. In this instance, space rays bring in \\$9 of profit and phasers bring in \\$4.50 of profit. We also need to go back to the original 1200 pounds of plastic.

In [51]:
# reset the rhs of the plastic constraint to 1200
print(f'Current RHS of plastic constraint is {plastic.rhs}')

plastic.rhs = 1200

# update the model
m.update()

print(f'After changing RHS of back to original is {plastic.rhs}')

Current RHS of plastic constraint is 1201.0
After changing RHS of back to original is 1200.0


In [52]:
# change the objective function
m.setObjective(9 * x_s + 4.5 * x_p)
m.update()
m.optimize()

# get the results
print(f'To generate the optimal profit of ${m.ObjVal:0.2f}, you should produce the following amounts:')
for v in m.getVars():
    print(f'   {v.VarName} = {v.X}')

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 24.04.2 LTS")

CPU model: Intel(R) Core(TM) Ultra 7 155H, instruction set [SSE2|AVX|AVX2]
Thread count: 11 physical cores, 22 logical processors, using up to 22 threads

Optimize a model with 4 rows, 2 columns and 8 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [4e+00, 9e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+02, 2e+03]
LP warm-start: use basis

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.4000000e+03   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  5.400000000e+03
To generate the optimal profit of $5400.00, you should produce the following amounts:
   space_rays = 480.0
   phasers = 240.0


In [53]:
# Get the reduced cost and range of optimality for each variable
get_SA_vars(m)

,variable,final_value,reduced_cost,obj_fn_coeff,range_opt_low,range_opt_up
0,space_rays,480.0,0.0,9.0,3.375,9.0
1,phasers,240.0,0.0,4.5,4.500,12.0


### How to Find Other Optimal Solutions

That answer only showed a single solution. We can ask Gurobi to find multiple solutions and then look at those.

In [54]:
# do a systematic search for the k-best solutions
m.setParam(GRB.Param.PoolSearchMode, 2)

# Update the model
m.update()

# Solve
m.optimize()

# Get the status
status = m.Status
if status in (GRB.INF_OR_UNBD, GRB.INFEASIBLE, GRB.UNBOUNDED):
    print('The model cannot be solved because it is infeasible or unbounded')

if status != GRB.OPTIMAL:
    print('Optimization was stopped with status ' + str(status))

if status == GRB.OPTIMAL:
    print(f'Optimization found an OPTIMAL answer with the status of {status}')

# Print number of solutions stored
nSolutions = m.SolCount
print('Number of solutions found: ' + str(nSolutions))

Set parameter PoolSearchMode to value 2
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 24.04.2 LTS")

CPU model: Intel(R) Core(TM) Ultra 7 155H, instruction set [SSE2|AVX|AVX2]
Thread count: 11 physical cores, 22 logical processors, using up to 22 threads

Non-default parameters:
PoolSearchMode  2

Optimize a model with 4 rows, 2 columns and 8 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [4e+00, 9e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+02, 2e+03]

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  5.400000000e+03
Optimization found an OPTIMAL answer with the status of 2
Number of solutions found: 1


#### Why Only 1 Solution?

When the variables are continuous and you have multiple optimal solutions, you will in fact have an **infinite** number of optimal solutions. Therefore, the solver will not try to search for more than one. Which one did it find?

In [55]:
for v in m.getVars():
    print(f'   {v.VarName} = {v.X}')

   space_rays = 480.0
   phasers = 240.0


#### Another Possible Approach?

Because the optimal solution found above is integer, you could try converting both variables to integer instead of continuous and re-solving the updated model. When dealing with integer or binary variables, Gurobi will try to find other optimal solutions.

In [56]:
# change the variable types to integer
for var in m.getVars():
    print(f'Variable {var.VarName} is currently of type {var.VTYPE}')
    var.setAttr(GRB.Attr.VType, GRB.INTEGER)

# update the model for changes to take effect
m.update()
print('After setting the variables to integer ...')
for var in m.getVars():
    print(f'Variable {var.VarName} is currently of type {var.VTYPE}')

Variable space_rays is currently of type C
Variable phasers is currently of type C
After setting the variables to integer ...
Variable space_rays is currently of type I
Variable phasers is currently of type I


In [57]:
# Solve
m.optimize()

# Print number of solutions stored
nSolutions = m.SolCount
print(f'Number of solutions found: {nSolutions}')

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 24.04.2 LTS")

CPU model: Intel(R) Core(TM) Ultra 7 155H, instruction set [SSE2|AVX|AVX2]
Thread count: 11 physical cores, 22 logical processors, using up to 22 threads

Non-default parameters:
PoolSearchMode  2

Optimize a model with 4 rows, 2 columns and 8 nonzeros
Model fingerprint: 0x55205400
Variable types: 0 continuous, 2 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [4e+00, 9e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+02, 2e+03]
Found heuristic solution: objective 5229.0000000
Presolve removed 1 rows and 0 columns
Presolve time: 0.00s
Presolved: 3 rows, 2 columns, 6 nonzeros
Variable types: 0 continuous, 2 integer (0 binary)

Root relaxation: objective 5.400000e+03, 1 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap 

In [58]:
# Print objective value and values of the decision varaibles of solutions
for soln_num in range(m.getAttr(GRB.Attr.SolCount)):
    m.Params.SolutionNumber = soln_num
    print(f'solution {soln_num} has obj fn value of {m.PoolObjVal}')
    for v in m.getVars():
        # Need to use v.Xn to the value of the variable v for this solution
        print(f'   {v.VarName} = {v.Xn}')

solution 0 has obj fn value of 5400.0
   space_rays = 550.0
   phasers = 100.0
solution 1 has obj fn value of 5400.0
   space_rays = 482.0
   phasers = 236.00000000000003
solution 2 has obj fn value of 5400.0
   space_rays = 516.0
   phasers = 168.0
solution 3 has obj fn value of 5400.0
   space_rays = 508.0
   phasers = 184.0
solution 4 has obj fn value of 5400.0
   space_rays = 542.0
   phasers = 116.0
solution 5 has obj fn value of 5400.0
   space_rays = 499.0
   phasers = 202.0
solution 6 has obj fn value of 5400.0
   space_rays = 491.0
   phasers = 218.0
solution 7 has obj fn value of 5400.0
   space_rays = 533.0
   phasers = 134.0
solution 8 has obj fn value of 5400.0
   space_rays = 525.0
   phasers = 150.0
solution 9 has obj fn value of 5400.0
   space_rays = 514.0
   phasers = 172.0


### Success!

When using integer variables, we can get multiple optimal solutions with Gurobi. Note that within in solution you need to use `v.Xn`, where `v` is the variable, to get the values of the decision variables.

**&copy; 2024 - Present: Matthew D. Dean, Ph.D.   
Clinical Full Professor of Business Analytics at William \& Mary.**